# AgentOps Basic Monitoring 
This is an example of how to use the AgentOps library for basic Agent monitoring with OpenAI's GPT

First let's install the required packages

In [1]:
%pip install -U openai
%pip install -U agentops
%pip install -U python-dotenv


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Then import them

In [2]:
from openai import OpenAI
import agentops
import os
from dotenv import load_dotenv

Next, we'll grab our API keys. You can use dotenv like below or however else you like to load environment variables

In [3]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or "<your_openai_key>"
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") or "<your_agentops_key>"

The AgentOps library is designed to be a plug-and-play replacement for the OpenAI Client, maximizing use with minimal install effort.

In [4]:
openai = OpenAI(api_key=OPENAI_API_KEY)
agentops.init(AGENTOPS_API_KEY, default_tags=["openai-gpt-notebook"])

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=037c5380-e7f2-4ab7-b85b-54b714416fc8


Now just use OpenAI as you would normally!

## Single Session with ChatCompletion

In [5]:
message = [{"role": "user", "content": "Write a 12 word poem about secret agents."}]
res = openai.chat.completions.create(
    model="gpt-3.5-turbo", messages=message, temperature=0.5, stream=False
)
print(res.choices[0].message["content"])

ChatCompletion(id='chatcmpl-9vfc9uAbQf2b4gkBLF8r2S1hemHff', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Silent shadows move, unseen, on covert missions of intrigue and espionage.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1723531689, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=15, prompt_tokens=17, total_tokens=32))


Make sure to end your session with a `Result` (Success|Fail|Indeterminate) for better tracking

In [6]:
agentops.end_session("Success")

🖇 AgentOps: This run's cost $0.000031
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=037c5380-e7f2-4ab7-b85b-54b714416fc8


Now if you check the AgentOps dashboard, you should see information related to this run!

# Events
Additionally, you can track custom events via AgentOps.
Let's start a new session and record some events 

In [7]:
# Create new session
agentops.start_session(tags=["openai-gpt-notebook-events"])

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=ed47e38d-39c7-498e-bccf-e4b5280c983d


The easiest way to record actions is through the use of AgentOp's decorators

In [8]:
from agentops import record_action


@record_action("add numbers")
def add(x, y):
    return x + y


add(2, 4)

6

We can also manually craft an event exactly the way we want

In [9]:
from agentops import ActionEvent

message = ({"role": "user", "content": "Hello"},)
response = openai.chat.completions.create(
    model="gpt-3.5-turbo", messages=message, temperature=0.5
)

if "hello" in str(response.choices[0].message.content).lower():
    agentops.record(
        ActionEvent(
            action_type="Agent says hello",
            params=str(message),
            returns=str(response.choices[0].message.content),
        )
    )

In [10]:
agentops.end_session("Success")

🖇 AgentOps: This run's cost $0.000017
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=ed47e38d-39c7-498e-bccf-e4b5280c983d
